In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import LSTM,Dense,Flatten,Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [2]:
df = pd.read_csv('/content/drive/MyDrive/file.csv')

In [ ]:
df.head()

,Unnamed: 0,tweets,labels
0,0,ChatGPT: Optimizing Language Models for Dialog...,neutral
1,1,"Try talking with ChatGPT, our new AI system wh...",good
2,2,ChatGPT: Optimizing Language Models for Dialog...,neutral
3,3,"THRILLED to share that ChatGPT, our new model ...",good
4,4,"As of 2 minutes ago, @OpenAI released their ne...",bad


In [3]:
df = df[['tweets', 'labels']]
df.head()

,tweets,labels
0,ChatGPT: Optimizing Language Models for Dialog...,neutral
1,"Try talking with ChatGPT, our new AI system wh...",good
2,ChatGPT: Optimizing Language Models for Dialog...,neutral
3,"THRILLED to share that ChatGPT, our new model ...",good
4,"As of 2 minutes ago, @OpenAI released their ne...",bad


In [4]:
# View the first 10 rows in the tweets column

for i in df.tweets.head(10):
    print(i)
    print()

ChatGPT: Optimizing Language Models for Dialogue https://t.co/K9rKRygYyn @OpenAI

Try talking with ChatGPT, our new AI system which is optimized for dialogue. Your feedback will help us improve it. https://t.co/sHDm57g3Kr

ChatGPT: Optimizing Language Models for Dialogue https://t.co/GLEbMoKN6w #AI #MachineLearning #DataScience #ArtificialIntelligence\n\nTrending AI/ML Article Identified &amp; Digested via Granola; a Machine-Driven RSS Bot by Ramsey Elbasheer https://t.co/RprmAXUp34

THRILLED to share that ChatGPT, our new model optimized for dialog, is now public, free, and accessible to everyone. https://t.co/dyvtHecYbd https://t.co/DdhzhqhCBX https://t.co/l8qTLure71

As of 2 minutes ago, @OpenAI released their new ChatGPT. \n\nAnd you can use it right now 👇 https://t.co/VyPGPNw988 https://t.co/cSn5h6h1M1

Just launched ChatGPT, our new AI system which is optimized for dialogue: https://t.co/ArX6m0FfLE.\n\nTry it out here: https://t.co/YM1gp5bA64

As of 2 minutes ago, @OpenAI release

In [5]:
# Remove all the tweet links since they all begin with https:

df['tweet_list'] = df['tweets'].str.split('https:')

In [ ]:
df.head()

,tweets,labels,tweet_list
0,ChatGPT: Optimizing Language Models for Dialog...,neutral,[ChatGPT: Optimizing Language Models for Dialo...
1,"Try talking with ChatGPT, our new AI system wh...",good,"[Try talking with ChatGPT, our new AI system w..."
2,ChatGPT: Optimizing Language Models for Dialog...,neutral,[ChatGPT: Optimizing Language Models for Dialo...
3,"THRILLED to share that ChatGPT, our new model ...",good,"[THRILLED to share that ChatGPT, our new model..."
4,"As of 2 minutes ago, @OpenAI released their ne...",bad,"[As of 2 minutes ago, @OpenAI released their n..."


In [6]:
# Select the text part of the list

text = [i[0] for i in df.tweet_list]

In [7]:
df['text'] = text

In [8]:
df = df[['text', 'labels']]

In [ ]:
df.head()

,text,labels
0,ChatGPT: Optimizing Language Models for Dialogue,neutral
1,"Try talking with ChatGPT, our new AI system wh...",good
2,ChatGPT: Optimizing Language Models for Dialogue,neutral
3,"THRILLED to share that ChatGPT, our new model ...",good
4,"As of 2 minutes ago, @OpenAI released their ne...",bad


In [9]:
# Import re for string processing

import re

In [10]:
# Remove all non-alphanumeric characters from the text list

string = r'[A-Za-z0-9 ]'

trim_list=[]

for row in text:
    s=''
    for letter in row:
        if bool(re.match(string, letter)):
            s+=letter
    trim_list.append(s)

In [11]:
# Remove the non-printing characters from text

rep_list = ['\U0001fae1', '\\n', '@', '#', '\xa0', '***']

for i in trim_list:
    for j in rep_list:
        if j in i:
            i.replace(j,'')

In [12]:
df['text'] = trim_list

In [13]:
df.head()

,text,labels
0,ChatGPT Optimizing Language Models for Dialogue,neutral
1,Try talking with ChatGPT our new AI system whi...,good
2,ChatGPT Optimizing Language Models for Dialogue,neutral
3,THRILLED to share that ChatGPT our new model o...,good
4,As of 2 minutes ago OpenAI released their new ...,bad


In [14]:
# Map the labels to integers
# 1 for good tweet
# 0 for neutral tweet
# -1 for bad tweet

df['lab_int'] = np.where(df['labels']=='good', 1, np.where(df['labels']=='bad', -1, 0))
df.lab_int

0         0
1         1
2         0
3         1
4        -1
         ..
219289   -1
219290    1
219291   -1
219292   -1
219293    0
Name: lab_int, Length: 219294, dtype: int64

In [26]:
tokenizer=Tokenizer(num_words=500)
tokenizer.fit_on_texts(df.text)

In [27]:
x_train,x_test,y_train,y_test = train_test_split(df.text,df.lab_int,test_size=0.3)

In [29]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from sklearn.metrics import classification_report

In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from sklearn.metrics import classification_report

# 텍스트 데이터 예시
texts = [
    'I love coding',
    'Machine learning is fascinating',
    'Natural language processing is important',
    'Coding is fun'
]

# 레이블 데이터 예시
labels = [1, 0, 2, 1]  # 예시에서는 삼진 분류 문제로 가정

# 텍스트 데이터를 정수로 인코딩
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
encoded_texts = tokenizer.texts_to_sequences(texts)

# 시퀀스 패딩
max_length = max([len(text.split()) for text in texts])  # 시퀀스의 최대 길이
padded_texts = pad_sequences(encoded_texts, maxlen=max_length, padding='post')

# 삼진 분류를 위해 레이블을 One-hot 인코딩
num_classes = len(set(labels))
one_hot_labels = tf.keras.utils.to_categorical(labels, num_classes)

# 데이터를 train과 test로 나누기
split_ratio = 0.8  # train 데이터의 비율
split_index = int(len(padded_texts) * split_ratio)

train_texts = padded_texts[:split_index]
train_labels = one_hot_labels[:split_index]

test_texts = padded_texts[split_index:]
test_labels = one_hot_labels[split_index:]

# RNN 모델 구축
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_length))
model.add(SimpleRNN(64))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
model.fit(train_texts, train_labels, epochs=10, batch_size=32)

# 테스트 데이터로 성능 측정
predictions = model.predict(test_texts)
predicted_labels = np.argmax(predictions, axis=1)

true_labels = np.argmax(test_labels, axis=1)

print(classification_report(true_labels, predicted_labels))







In [19]:
vocab=len(tokenizer.word_index)
vocab

162587

In [25]:
max_words = 10000
max_len = 500

In [ ]:
tokenizer=Tokenizer(num_words=500)
tokenizer.fit_on_texts(x_train)

In [ ]:
# Get our training data word index
vocab=len(tokenizer.word_index)
vocab

127413

In [30]:
sequences = tokenizer.texts_to_sequences(x_train)
test_sequences=tokenizer.texts_to_sequences(x_test)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)
sequences_matrix[:5]

array([[  0,   0,   0, ...,   7,   1,  29],
       [  0,   0,   0, ...,   1, 478,  51],
       [  0,   0,   0, ...,   0,   1,   6],
       [  0,   0,   0, ...,  23,   3,  55],
       [  0,   0,   0, ...,   2, 462,   2]], dtype=int32)

In [ ]:

# Get our training data word index
vocab=len(tokenizer.word_index)
vocab

127413

In [ ]:
max_len=max([len(i) for i in sequences])
max_len

54

In [21]:
from keras.layers import Embedding
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

In [ ]:
model=Sequential()
model.add(Embedding(vocab,100,input_length=54))
model.add(LSTM(128,return_sequences=False,activation="relu"))
model.add(Dense(2,activation="softmax"))

model.compile(loss='categorical_crossentropy',optimizer ='adam',metrics=['accuracy'])

In [ ]:
x=np.array(sequences_matrix)
y=y_train.values

In [ ]:
model.fit(x,y,epochs=10,verbose=2)

Epoch 1/10


ValueError: ignored

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
history = model.fit(x, y,
epochs=10,
batch_size=128,
)

Epoch 1/10


ValueError: ignored